# Extract entities and relations from a set of databases

In [4]:
### Step1: Extract entities
bash scripts/extract_entities.sh

Extracting entities from the ctdbase
2023-08-01 21:08:14,800 Found headers ['AnatomyName', 'AnatomyID', 'Definition', 'AltAnatomyIDs', 'ParentIDs', 'TreeNumbers', 'ParentTreeNumbers', 'Synonyms', 'ExternalSynonyms'] in databases/ctd/CTD_anatomy.csv
2023-08-01 21:08:14,802 Read 1844 rows from databases/ctd/CTD_anatomy.csv.
2023-08-01 21:08:14,803 Extract 1844 valid entities from databases/ctd/CTD_anatomy.csv...
2023-08-01 21:08:14,804 Drop duplicates, and 1844 entities are left.
2023-08-01 21:08:14,807 Done! The extracted entities are saved in extracted_entities/raw_entities/ctd/ctd_anatomy.tsv.

2023-08-01 21:08:14,807 Found headers ['ChemicalName', 'ChemicalID', 'CasRN', 'Definition', 'ParentIDs', 'TreeNumbers', 'ParentTreeNumbers', 'Synonyms'] in databases/ctd/CTD_chemicals.csv
2023-08-01 21:08:14,827 Read 175910 rows from databases/ctd/CTD_chemicals.csv.
2023-08-01 21:08:14,844 Extract 175910 valid entities from databases/ctd/CTD_chemicals.csv...
2023-08-01 21:08:14,852 Drop duplica

In [13]:
### Step2: Merge entities
dot_clean .
mkdir -p extracted_entities/merged_entities
python scripts/merge_entities.py from-databases -i extracted_entities/raw_entities -o extracted_entities/merged_entities

Resources: ['extracted_entities/raw_entities/ctd/ctd_anatomy.tsv', 'extracted_entities/raw_entities/ctd/ctd_compound.tsv', 'extracted_entities/raw_entities/ctd/ctd_gene.tsv', 'extracted_entities/raw_entities/ctd/ctd_disease.tsv', 'extracted_entities/raw_entities/ctd/ctd_pathway.tsv', 'extracted_entities/raw_entities/ctd/ctd_biological_process.tsv', 'extracted_entities/raw_entities/ctd/ctd_cellular_component.tsv', 'extracted_entities/raw_entities/ctd/ctd_molecular_function.tsv', 'extracted_entities/raw_entities/hetionet/hetionet_anatomy.tsv', 'extracted_entities/raw_entities/hetionet/hetionet_biological_process.tsv', 'extracted_entities/raw_entities/hetionet/hetionet_cellular_component.tsv', 'extracted_entities/raw_entities/hetionet/hetionet_compound.tsv', 'extracted_entities/raw_entities/hetionet/hetionet_disease.tsv', 'extracted_entities/raw_entities/hetionet/hetionet_gene.tsv', 'extracted_entities/raw_entities/hetionet/hetionet_molecular_function.tsv', 'extracted_entities/raw_entitie

In [14]:
### Step3: Format entities
# For disease
onto-match ontology -i extracted_entities/merged_entities/disease.tsv -o formatted_entities/disease.tsv -O disease
## Keep all duplicated rows
awk -F'\t' 'NR > 1 { count[$1]++ } END { for (item in count) if (count[item] > 1) print item }' formatted_entities/disease.tsv > formatted_entities/disease.duplicated.tsv
## Deduplicate rows by id.
awk -F'\t' 'NR == 1 || !seen[$1]++' formatted_entities/disease.tsv > formatted_entities/disease.filtered.tsv

# For gene
onto-match ontology -i extracted_entities/merged_entities/gene.tsv -o formatted_entities/gene.tsv -O gene -s 5 -b 500
awk -F'\t' 'NR == 1 || ($8 == 10090 || $8 == 9606)' formatted_entities/gene.tsv > formatted_entities/gene.filtered.tsv
## Deduplicate rows by id. The following processes are not necessary at most time.
awk -F'\t' 'NR == 1 || !seen[$1]++' formatted_entities/gene.filtered.tsv > formatted_entities/tmp_gene.filtered.tsv
mv formatted_entities/tmp_gene.filtered.tsv formatted_entities/gene.filtered.tsv

# For compound
onto-match ontology -i extracted_entities/merged_entities/compound.tsv -o formatted_entities/compound.tsv -O compound -s 5 -b 500
## Deduplicate rows by id. The following processes are not necessary at most time.
awk -F'\t' 'NR == 1 || !seen[$1]++' formatted_entities/compound.tsv > formatted_entities/compound.filtered.tsv

# For metabolite
onto-match ontology -i extracted_entities/merged_entities/metabolite.tsv -o formatted_entities/metabolite.tsv -O metabolite -s 5 -b 500
## Deduplicate rows by id. The following processes are not necessary at most time.
awk -F'\t' 'NR == 1 || !seen[$1]++' formatted_entities/metabolite.tsv > formatted_entities/metabolite.filtered.tsv

# For pathway
cp extracted_entities/merged_entities/pathway.tsv formatted_entities/pathway.tsv

# For side-effect
cp extracted_entities/merged_entities/side_effect.tsv formatted_entities/side_effect.tsv

# For symptom
cp extracted_entities/merged_entities/symptom.tsv formatted_entities/symptom.tsv

# For anatomy
cp extracted_entities/merged_entities/anatomy.tsv formatted_entities/anatomy.tsv

# For cellular_component
cp extracted_entities/merged_entities/cellular_component.tsv formatted_entities/cellular_component.tsv

# For biological_process
cp extracted_entities/merged_entities/biological_process.tsv formatted_entities/biological_process.tsv

# For molecular_function
cp extracted_entities/merged_entities/molecular_function.tsv formatted_entities/molecular_function.tsv

# For pharmacologic_class
cp extracted_entities/merged_entities/pharmacologic_class.tsv formatted_entities/pharmacologic_class.tsv

2023-08-01 21:38:01 - ontology_matcher.cli:72 - INFO - Enable the cache, you can use --disable-cache to disable it.
2023-08-01 21:38:01 - ontology_matcher.cli:74 - INFO - The cache file is ./ontology_matcher_cache.sqlite, if you encounter any problem, you can delete it or disable cache and rerun the command.
2023-08-01 21:38:01 - ontology_matcher.cli:98 - INFO - The json file already exists, if you want to reformat, please add --reformat flag or delete the output file and the json file, then rerun the command.
2023-08-01 21:38:02 - ontology_matcher.cli:72 - INFO - Enable the cache, you can use --disable-cache to disable it.
2023-08-01 21:38:02 - ontology_matcher.cli:74 - INFO - The cache file is ./ontology_matcher_cache.sqlite, if you encounter any problem, you can delete it or disable cache and rerun the command.
2023-08-01 21:38:02 - ontology_matcher.cli:98 - INFO - The json file already exists, if you want to reformat, please add --reformat flag or delete the output file and the jso

In [15]:
### Step4: Merge entity files into one file
dot_clean .
mkdir -p graph_data
python scripts/merge_entities.py to-single-file -i formatted_entities -o graph_data/entities.tsv

Entity files: ['formatted_entities/compound.tsv', 'formatted_entities/gene.tsv', 'formatted_entities/disease.tsv', 'formatted_entities/disease.duplicated.tsv', 'formatted_entities/metabolite.tsv', 'formatted_entities/metabolite.filtered.tsv', 'formatted_entities/compound.filtered.tsv', 'formatted_entities/pathway.tsv', 'formatted_entities/side_effect.tsv', 'formatted_entities/symptom.tsv', 'formatted_entities/anatomy.tsv', 'formatted_entities/cellular_component.tsv', 'formatted_entities/biological_process.tsv', 'formatted_entities/molecular_function.tsv', 'formatted_entities/pharmacologic_class.tsv', 'formatted_entities/disease.filtered.tsv', 'formatted_entities/gene.filtered.tsv']

Entity types:['compound', 'gene', 'disease', 'disease', 'metabolite', 'metabolite', 'compound', 'pathway', 'side_effect', 'symptom', 'anatomy', 'cellular_component', 'biological_process', 'molecular_function', 'pharmacologic_class', 'disease', 'gene']

Grouped entity files: {'compound': 'formatted_entities/

In [ ]:
### Step5: Extract relations from a set of databases
graph-builder --database ctd --database drkg --database primekg --database hsdn -d ./graph_data/relations -o ./graph_data/formatted_relations -f ./graph_data/entities.tsv -n 20 --download --skip -l ./graph_data/log.txt --debug

2023-08-01 21:39:32 - cli:105 - INFO - Run jobs with (output_dir: ./graph_data/formatted_relations, db_dir: ./graph_data/relations, databases: ('ctd', 'drkg', 'primekg', 'hsdn'), download: True, skip: True)
2023-08-01 21:39:32 - base_parser:268 - INFO - Download file from http://ctdbase.org/reports/CTD_chem_gene_ixns.tsv.gz into graph_data/relations/ctd/CTD_chem_gene_ixns.tsv.gz
2023-08-01 21:39:32 - base_parser:271 - INFO - graph_data/relations/ctd/CTD_chem_gene_ixns.tsv.gz exists, don't need to download it.
2023-08-01 21:39:32 - base_parser:268 - INFO - Download file from http://ctdbase.org/reports/CTD_chemicals_diseases.tsv.gz into graph_data/relations/ctd/CTD_chemicals_diseases.tsv.gz
2023-08-01 21:39:32 - base_parser:271 - INFO - graph_data/relations/ctd/CTD_chemicals_diseases.tsv.gz exists, don't need to download it.
2023-08-01 21:39:32 - base_parser:268 - INFO - Download file from http://ctdbase.org/reports/CTD_chem_pathways_enriched.tsv.gz into graph_data/relations/ctd/CTD_chem

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



2023-08-01 21:59:02 - base_parser:408 - INFO - Processing 19.46%
2023-08-01 21:59:02 - base_parser:408 - INFO - Processing 19.46%
2023-08-01 21:59:02 - base_parser:408 - INFO - Processing 19.46%
2023-08-01 21:59:02 - base_parser:408 - INFO - Processing 19.46%
2023-08-01 21:59:02 - base_parser:408 - INFO - Processing 19.47%
2023-08-01 21:59:02 - base_parser:408 - INFO - Processing 19.47%
2023-08-01 21:59:02 - base_parser:408 - INFO - Processing 19.47%
2023-08-01 21:59:02 - base_parser:408 - INFO - Processing 19.47%
2023-08-01 21:59:02 - base_parser:408 - INFO - Processing 19.47%
2023-08-01 21:59:02 - base_parser:408 - INFO - Processing 19.47%
2023-08-01 21:59:02 - base_parser:408 - INFO - Processing 19.47%
2023-08-01 21:59:02 - base_parser:408 - INFO - Processing 19.47%
2023-08-01 21:59:02 - base_parser:408 - INFO - Processing 19.48%
2023-08-01 21:59:02 - base_parser:408 - INFO - Processing 19.48%
2023-08-01 21:59:02 - base_parser:408 - INFO - Processing 19.48%
2023-08-01 21:59:02 - bas

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



2023-08-01 21:59:05 - base_parser:408 - INFO - Processing 46.05%
2023-08-01 21:59:05 - base_parser:408 - INFO - Processing 46.05%
2023-08-01 21:59:05 - base_parser:408 - INFO - Processing 46.05%
2023-08-01 21:59:05 - base_parser:408 - INFO - Processing 46.05%
2023-08-01 21:59:05 - base_parser:408 - INFO - Processing 46.06%
2023-08-01 21:59:05 - base_parser:408 - INFO - Processing 46.06%
2023-08-01 21:59:05 - base_parser:408 - INFO - Processing 46.06%
2023-08-01 21:59:05 - base_parser:408 - INFO - Processing 46.06%
2023-08-01 21:59:05 - base_parser:408 - INFO - Processing 46.06%
2023-08-01 21:59:05 - base_parser:408 - INFO - Processing 46.06%
2023-08-01 21:59:05 - base_parser:408 - INFO - Processing 46.06%
2023-08-01 21:59:05 - base_parser:408 - INFO - Processing 46.06%
2023-08-01 21:59:05 - base_parser:408 - INFO - Processing 46.07%
2023-08-01 21:59:05 - base_parser:408 - INFO - Processing 46.07%
2023-08-01 21:59:05 - base_parser:408 - INFO - Processing 46.07%
2023-08-01 21:59:05 - bas